In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv
from slack_bolt import App
from slack_bolt.adapter.socket_mode import SocketModeHandler

In [2]:
load_dotenv('../.env')
SLACK_BOT_TOKEN = os.getenv("SLACK-BOT-RECEIPT")
CHANNEL_ID = 'C086KU2133M' # Channel ID
SLACK_APP_TOKEN = os.getenv("SLACK-APP-RECEIPT")

In [3]:
# Slack 앱을 초기화합니다
app = App(token=SLACK_BOT_TOKEN)

In [5]:
# 모든 이벤트를 로깅하는 미들웨어
@app.middleware
def log_request(logger, body, next):
    print(f"이벤트 수신: {body['event']['type']}")
    next()

In [6]:
# 메시지 이벤트를 처리하는 리스너
@app.message()
def handle_message(message, logger, say):
    # 메시지 내용을 가져옵니다
    text = message['text']
    print(f"메시지를 받았습니다: {text}")
    # 메시지에 대한 응답을 보냅니다
    say(f"메시지를 받았습니다: {text}", channel=CHANNEL_ID)

In [7]:
# 에러 핸들러
@app.error
def custom_error_handler(error, body, logger):
    print(f"에러 발생: {error}")

In [ ]:
handler = SocketModeHandler(app, SLACK_APP_TOKEN)
handler.start()

### 채널로부터 이미지 가지고 오기

In [9]:
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from slack_sdk.socket_mode import SocketModeClient
from slack_sdk.socket_mode.request import SocketModeRequest
from dotenv import load_dotenv

client = WebClient(token=SLACK_BOT_TOKEN)
socket_client = SocketModeClient(app_token=SLACK_APP_TOKEN, web_client=client)

In [10]:
def fetch_images_from_channel(channel_id):
    """
    주어진 Slack 채널에서 이미지를 가져옵니다.
    Args:
        channel_id (str): Slack 채널 ID.
    Returns:
        list: 이미지 파일 URL 목록.
    """
    try:
        # 채널 메시지 가져오기
        print('try..')
        response = client.conversations_history(channel=channel_id)
        print(f'response = {response}')
        messages = response['messages']
        
        image_urls = []
        
        # 메시지 내 파일 확인
        for message in messages:
            if 'files' in message:  # 파일이 포함된 메시지인지 확인
                for file in message['files']:
                    if file['mimetype'].startswith('image/'):  # 이미지 파일만 필터링
                        image_urls.append(file['url_private'])

        return image_urls

    except SlackApiError as e:
        print(f"Slack API 오류: {e.response['error']}")
        return []

In [ ]:
# 이미지 가져오기
image_urls = fetch_images_from_channel(CHANNEL_ID)

# 결과 출력
print("이미지 URL 목록:")
for url in image_urls:
    print(url)